In [1]:
!date

Tue Jan 20 02:45:50 PM EST 2026


In [2]:
pwd

'/projects/sccn/andromeda1/aglinska/deepcor-fmri-toolbox/examples'

In [3]:
import os
from pathlib import Path

bids_path = '../Data/fMRI-Data/studyforrest-fmriprep/'
sub_id = 'sub-01'
session = 'ses-localizer'
task = 'objectcategories'
run = '1'
space = 'MNI152NLin2009cAsym'

In [4]:
pwd

'/projects/sccn/andromeda1/aglinska/deepcor-fmri-toolbox/examples'

In [5]:
# Construct paths
base = os.path.join(bids_path,sub_id,session)

# EPI
epi_path = os.path.join(base,'func',f'{sub_id}_{session}_task-{task}_run-{run}_bold_space-{space}_preproc.nii.gz')

# Confounds
confounds_path = os.path.join(base,'func',f'{sub_id}_{session}_task-{task}_run-{run}_bold_confounds.tsv')

gm_mask_path = os.path.join(bids_path,'mask_roi.nii')
cf_mask_path = os.path.join(bids_path,'mask_roni.nii')

assert os.path.exists(epi_path), 'epi_path does not exist'
assert os.path.exists(confounds_path), 'confounds_path does not exist'
assert os.path.exists(gm_mask_path), 'gm_mask_path does not exist'
assert os.path.exists(cf_mask_path), 'cf_mask_path does not exist'

output_dir = os.path.join('../Data/DeepCor-Outputs',f'DeepCor-Forrest-{sub_id}-{task}-{run}-cvae_v1')

print("EPI:", epi_path)
print("Confounds:", confounds_path)
print("output_dir:", output_dir)

EPI: ../Data/fMRI-Data/studyforrest-fmriprep/sub-01/ses-localizer/func/sub-01_ses-localizer_task-objectcategories_run-1_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
Confounds: ../Data/fMRI-Data/studyforrest-fmriprep/sub-01/ses-localizer/func/sub-01_ses-localizer_task-objectcategories_run-1_bold_confounds.tsv
output_dir: ../Data/DeepCor-Outputs/DeepCor-Forrest-sub-01-objectcategories-1-cvae_v1


In [6]:
import torch
from deepcor import DeepCorDenoiser

# ---------------------------
# Run DeepCor (simplified API)
# ---------------------------

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# model_version:
# - "v1": original CVAE
# - "v2": current CVAE (default / latest)
# - "latest": alias for the current recommended model

denoiser = DeepCorDenoiser(
    model_version="v1",
    device=device, # you can tweak these, but defaults are fine for a first run
    n_epochs=100,           # keep small for a quick sanity run; increase for real denoising
    n_repetitions=20,      # keep small for a quick sanity run; increase for real denoising
    batch_size=1024,
    latent_dims=(8, 8),   # for v1 only the first value is used
    verbose=True,
)

output_path = denoiser.fit_denoise(
    epi_path=str(epi_path),
    gm_mask_path=str(gm_mask_path),
    cf_mask_path=str(cf_mask_path),
    confounds_path=str(confounds_path),
    output_dir=str(output_dir),
    verbose=True,
)

print("\nDone. Denoised file:", output_path)

Using device: cuda:0
device is cuda:0
Loading data...
Preparing training data...
obs_list.shape: (41116, 156)
noi_list.shape: (9092, 156)
upsampling noi_list_coords
obs_list.shape: (41116, 156)
noi_list.shape: (41116, 156)


Training repetition 2:   5%|▉                 | 1/20 [08:40<2:44:54, 520.74s/it]


KeyboardInterrupt: 

In [ ]:
from deepcor.analysis import get_design_matrix, run_correlation_analysis_from_spec, run_contrast_analysis_from_spec

In [ ]:
#!cp -r ../../BC-ABCD-denoise/Data/StudyForrest/events ../Data/
events_path = f'../Data/study-forrest-events/{sub_id}_ses-localizer_task-objectcategories_run-{run}_events.tsv'
assert os.path.exists(events_path),'events_path does not exist'

import ants
epi = ants.image_read(epi_path)
X1 = get_design_matrix(epi,events_path)

In [ ]:
#!cp -r ../../BC-ABCD-denoise/Data/StudyForrest/ROIs/ ../Data/

correlation_analyses = []
contrast_analyses = []

correlation_analyses.append(
    {'corr_target' : X1['face'].values,
     'filename' : os.path.join(output_dir,'corr2face.nii.gz'),
     'plot' : True, 
     'ROI' : f'../Data/study-forrest-ROIs/rFFA_final_mask_{sub_id}_bin.nii.gz'})

correlation_analyses.append(
    {'corr_target' : X1[['house','scene']].values.mean(axis=1),
     'filename' : os.path.join(output_dir,'corr2place.nii.gz'),
     'plot' : True,
     'ROI' : f'../Data/study-forrest-ROIs/rPPA_final_mask_{sub_id}_bin.nii.gz'})

contrast_analyses.append(
    {'contrast_vec' : [-1,5,-1,-1,-1,-1,0,0,0,0],
     'design_matrix' : X1,
     'filename' : os.path.join(output_dir,'contrast_face.nii.gz'),
     'plot' : True, 
     'ROI' : f'../Data/study-forrest-ROIs/rFFA_final_mask_{sub_id}_bin.nii.gz'})

contrast_analyses.append(
    {'contrast_vec' : [-1,-1,2,-1,2,-1,0,0,0,0],
     'design_matrix' : X1,
     'filename' : os.path.join(output_dir,'contrast_place.nii.gz'),
     'plot' : True,
     'ROI' : f'../Data/study-forrest-ROIs/rPPA_final_mask_{sub_id}_bin.nii.gz'})

In [ ]:
signals_averaged = ants.image_read(os.path.join(output_dir,'denoised_deepcor.nii.gz'))
compcor = ants.image_read(os.path.join(output_dir,'denoised_compcor.nii.gz'))
#epi = ants.image_read(os.path.join(output_dir,'denoised_deepcor.nii.gz'))
gm = ants.image_read(gm_mask_path)

In [ ]:
run_correlation_analysis_from_spec(correlation_analyses[0], epi, compcor, signals_averaged,gm)
run_correlation_analysis_from_spec(correlation_analyses[1], epi, compcor, signals_averaged,gm)

In [ ]:
run_contrast_analysis_from_spec(contrast_analyses[0], epi, compcor, signals_averaged,gm)
run_contrast_analysis_from_spec(contrast_analyses[1], epi, compcor, signals_averaged,gm)